In [1]:
import os

data_dir = "../BigEarthNet-v1.0/"
cloud_shadow_file = "../patches_with_cloud_and_shadow.csv"
snow_file = "../patches_with_seasonal_snow.csv"
patches = os.listdir(data_dir)
patches.sort()
len(patches)

%load_ext autoreload
%autoreload 2

import load_data

In [4]:
data = load_data.BigEarthNetDataset(split_file="splits.pkl")

Reloading train-val-test split cache from splits.pkl


In [23]:
meta_dataset = load_data.MetaBigEarthNetTaskDataset(split_file="splits.pkl")

Reloading train-val-test split cache from splits.pkl


In [34]:
imgs, labels = next(iter(meta_dataset))

In [35]:
labels.size()

torch.Size([4, 43])

In [ ]:
import csv
import gdal
import rasterio

In [ ]:

elimination_patch_list = []  
for file_path in [cloud_shadow_file, snow_file]:
    if not os.path.exists(file_path):
        print('ERROR: file located at', file_path, 'does not exist')
        exit()
    with open(file_path, 'r') as f:
        csv_reader = csv.reader(f, delimiter=',')
        for row in csv_reader:
            elimination_patch_list.append(row[0])
#print('INFO:', len(elimination_patch_list), 'number of patches will be eliminated')
elimination_patch_list = set(elimination_patch_list)


In [ ]:
filtered_patches = [patch for patch in patches if patch not in elimination_patch_list]
len(filtered_patches)

In [ ]:
os.listdir(os.path.join(data_dir, patches[2]))

In [ ]:
import random
idx = random.randint(0, len(patches))
all_bands = ['B01', 'B02', 'B03', 'B04', 'B05',
              'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']
rgb_bands = ['B04', 'B03', 'B02']

band_stack = []
for bands in rgb_bands:
    band_path = os.path.join(data_dir, patches[idx], "{}_{}.tif".format(patches[idx], bands))
    print("Loading", band_path)
    assert os.path.isfile(band_path)

    band_ds = gdal.Open(band_path,  gdal.GA_ReadOnly)
    raster_band = band_ds.GetRasterBand(1)
    band_data = raster_band.ReadAsArray()
    band_stack.append(band_data)



In [ ]:
import matplotlib.pyplot as plt

_OPTICAL_MAX_VALUE = 2000. # magic number from some google guys
img = np.stack(band_stack) / _OPTICAL_MAX_VALUE # (C, W, H)
img = np.clip(img, 0, 1)
plt.imshow(np.transpose(img))


In [ ]:
import json

In [ ]:
with open(os.path.join(data_dir, patches[idx], "{}_labels_metadata.json".format(patches[idx])), 'r') as f:
    metadata = json.load(f)

metadata

In [ ]:
from collections import Counter

c = Counter()
c.update(metadata['labels'])
c

In [ ]:
from collections import Counter
from tqdm.notebook import tqdm
import pickle

label_counts = Counter()
for patch in tqdm(patches):
    with open(os.path.join(data_dir, patch, "{}_labels_metadata.json".format(patch)), 'r') as f:
        metadata = json.load(f)
        label_counts.update(metadata['labels'])
print(label_counts)

with open("label_counts_cache.pkl", "wb") as f:
    pickle.dump(label_counts, f)

In [ ]:
random.choice(label_counts.keys(), 2)